Project:
<ul>
<li> There are two Uniswap Pools, Pool A and Pool B. </li>
<li> Pool A has X1 DAI and Y1 ETH, Pool B has X2 DAI and Y2 ETH. </li>
<li> Swap fee is 0.3%.</li>
</ul>
As the constant product formula mentioned before,
<ul>
<li> X1 * Y1 = K1 </li>
<li> X2 * Y2 = K2 </li>
</ul>
The ETH-DAI relative price of Pool A is Y1 /X1 , and for Pool B it is Y2 /X2. Because the relative price of the two pair assets can only be changed through trading, divergences between two pools create arbitrage opportunities. Please write a computer program (Rust/C++/GO preferred, but can use any programming language you are most familiar with) with the following requirements:

<b> 1. Define the data structs of pool status, and implement add/remove/swap functions to maintain status data in memory.

In [2]:
import math
import asyncio
import time

In [1]:
class PoolStatus:
    def __init__(self, name, dai, eth):
        self.name = name
        self.dai = dai
        self.eth = eth
        self.k = self.eth * self.dai
        self.price = self.eth / self.dai    # price of dai in terms of eth
    
    def getPrice(self):
        return self.price

    def add(self, dai=0, eth=0):
        self.dai += dai
        self.eth += eth
        self.k = self.dai * self.eth
        self.price = self.eth / self.dai

    def remove(self, dai=0, eth=0):
        self.dai -= dai
        self.eth -= eth
        self.k = self.dai * self.eth
        self.price = self.eth / self.dai

    def getPriceInput(self, output_dai=0, output_eth=0):
        if output_eth:
            return output_eth / self.price * 1.003
        elif output_dai:
            return output_dai * self.price * 1.003

    def getPriceOutput(self, input_dai=0, input_eth=0):  # eg: returns the amount of dai if input 1 eth
        if input_eth:
            return self.dai - self.k / (input_eth + self.eth)
        elif input_dai:
            return self.eth - self.k / (input_dai + self.dai)
    
    def swap(self, input_dai=0, input_eth=0):   # eg: user wants to swap dai for eth, so our pool increases in dai, decreases in eth
        if input_eth:
            remove_dai = self.getPriceOutput(input_eth=input_eth)
            self.remove(dai=remove_dai)
            add_eth = input_eth * 1.003
            self.add(eth=add_eth)
        elif input_dai:
            remove_eth = self.getPriceOutput(input_dai=input_dai)
            self.remove(eth=remove_eth)
            add_dai = input_dai * 1.003
            self.add(dai=add_dai)
    
    def check(self):
        print(f'Pool {self.name} status')
        print(f'DAI: {self.dai}')
        print(f'ETH: {self.eth}')
        print(f'Price: {self.price}')
        print(f'k: {self.k}\n')
        
        
def main():
    pool_a = PoolStatus('A', 400, 1200)
    pool_b = PoolStatus('B', 200, 20)

    pool_a.check()
    pool_b.check()

    print(f'To get 1 DAI, required amount of ETH to swap: {pool_a.getPriceInput(output_dai=1)}')
    print(f'Amount of DAI you will receive if you swap 3 ETH: {pool_a.getPriceOutput(input_eth=3)}\n')

    pool_a.swap(input_dai=3)
    print(f'===== Swapped =====')
    pool_a.check()
    

if __name__ == "__main__":
    main()

Pool A status
DAI: 400
ETH: 1200
Price: 3.0
k: 480000

Pool B status
DAI: 200
ETH: 20
Price: 0.1
k: 4000

To get 1 DAI, required amount of ETH to swap: 3.0089999999999995
Amount of DAI you will receive if you swap 3 ETH: 0.9975062344139474

===== Swapped =====
Pool A status
DAI: 403.009
ETH: 1191.0669975186104
Price: 2.955435232261836
k: 480010.7196029777



<b> 2. Define the arbitrage calculation logic, including how to trigger the calculation in non-blocking mode, find out how many ETH do we need for an arbitrage transaction and how much profit(in ETH) can be made

Arbitrage calculation logic:
<ol>
<li> Find out the ETH-DAI price from both pools. </li>
<li> Get the ETH required from the lower-priced pool to get X amount of DAI. </li>
<li> Get the ETH received from the higher-priced pool when we swap X amount of DAI. </li>
<li> After accounting 0.3% fees for each transaction, get the profit. </li>
<li> If profit exists, return amount of ETH required for trading X amount of DAI. </li>
</ol>
<br> The arbitrage calculation can be triggered in non-blocking mode by using an asynchronous function. Thus, the main thread will not be blocked while we are retrieving and calculating of prices of both pools on separate threads.

In [4]:
class PoolStatus:
    def __init__(self, name, dai, eth):
        self.name = name
        self.dai = dai
        self.eth = eth
        self.k = self.eth * self.dai
        self.price = self.eth / self.dai    # price of dai in terms of eth

    def getName(self):
        return self.name
    
    async def getPrice(self):
        return self.price

    async def add(self, dai=0, eth=0):
        self.dai += dai
        self.eth += eth
        self.k = self.dai * self.eth
        self.price = self.eth / self.dai

    async def remove(self, dai=0, eth=0):
        self.dai -= dai
        self.eth -= eth
        self.k = self.dai * self.eth
        self.price = self.eth / self.dai

    def getPriceInput(self, output_dai=0, output_eth=0):
        if output_eth:
            return (self.k / (self.eth - output_eth) - self.dai) * 1.003
        elif output_dai:
            return (self.k / (self.dai - output_dai) - self.eth) * 1.003

    def getPriceOutput(self, input_dai=0, input_eth=0):  # eg: returns the amount of dai if input 1 eth
        if input_eth:
            return self.dai - self.k / (input_eth + self.eth)
        elif input_dai:
            return self.eth - self.k / (input_dai + self.dai)
    
    async def swap(self, input_dai=0, input_eth=0):   # eg: user wants to swap dai for eth, so our pool increases in dai, decreases in eth
        if input_eth:
            remove_dai = self.getPriceOutput(input_eth=input_eth)
            await self.remove(dai=remove_dai)
            add_eth = input_eth * 1.003
            await self.add(eth=add_eth)
        elif input_dai:
            remove_eth = self.getPriceOutput(input_dai=input_dai)
            await self.remove(eth=remove_eth)
            add_dai = input_dai * 1.003
            await self.add(dai=add_dai)

    def check(self):
        print(f'Pool {self.name} status')
        print(f'DAI: {round(self.dai, 6)}')
        print(f'ETH: {round(self.eth, 6)}')
        print(f'Price: {round(self.price, 6)}')
        print(f'k: {round(self.k, 6)}\n')

In [6]:
## RUN PREVIOUS CELL FIRST

async def calculate_arbitrage(pool_a, pool_b, output_dai):
    # Get the current ETH-DAI price from Pool A and Pool B.
    price_a = await pool_a.getPrice()
    price_b = await pool_b.getPrice()

    # Calculate the relative price of ETH-DAI between Pool A and Pool B.
    if price_a > price_b:
        higher, lower = pool_a, pool_b
    elif price_b > price_a:
        higher, lower = pool_b, pool_a
    else:
        return None
    
    required_eth = lower.getPriceInput(output_dai=output_dai)    # amount of ETH to swap for 1 DAI
    received_eth = higher.getPriceOutput(input_dai=output_dai)   # amount of ETH received after returning 1 DAI in other pool
    profit = received_eth - required_eth

    if profit > 0:
        return required_eth, profit, lower.getName()
    else:
        return None

def opportunityStatus(opp=None, output_dai=1):
    if opp:
        print('--- Arbitrage opportunity exists ---')
        print(f'Using {output_dai} DAI:')
        print(f'Buy {round(opp[0], 6)} ETH from Pool {opp[2]}')
        print(f'Price per DAI: {round(opp[0]/output_dai, 6)} ETH')
        print(f'Total price: {round(opp[0], 6)} ETH')
        print(f'Total profit: ${round(opp[1], 6)}\n')
    else:
        print('--- No arbitrage opportunity ---\n')


pool_a = PoolStatus('A', 400, 1200)
pool_b = PoolStatus('B', 2000, 5800)

pool_a.check()
pool_b.check()

opportunity = await calculate_arbitrage(pool_a, pool_b, output_dai=10)
opportunityStatus(opp=opportunity, output_dai=10)

Pool A status
DAI: 400
ETH: 1200
Price: 3.0
k: 480000

Pool B status
DAI: 2000
ETH: 5800
Price: 2.9
k: 11600000

--- Arbitrage opportunity exists ---
Using 10 DAI:
Buy 29.233166 ETH from Pool B
Price per DAI: 2.923317 ETH
Total price: 29.233166 ETH
Total profit: $0.035127



<b> 3. Write test cases to simulate user’s swap transaction (eg. Swap 100 DAI to ETH in Pool A)

In [243]:
pool_a = PoolStatus('A', 400, 1200)
pool_b = PoolStatus('B', 2000, 6000)

pool_a.check()
pool_b.check()

## Test Case 1: Swap 100 DAI to ETH in Pool A
# Input: 100 DAI
# Expected output: DAI + 100.3 | ETH - 240

print('Test Case 1')
await pool_a.swap(input_dai=100)
print('===== Swapped =====')
pool_a.check()

## Test Case 2: Swap 200 ETH to DAI in Pool A
# Input: 200 ETH
# Expected output: ETH + 200.6 | DAI - 86.26

print('Test Case 2')
await pool_a.swap(input_eth=200)
print(f'===== Swapped =====')
pool_a.check()

## Test Case 3: Swap 50 ETH to DAI in Pool B
# Input: 50 ETH
# Expected output: ETH + 50.15 | ETH - 16.53

print('Test Case 3')
await pool_b.swap(input_eth=50)
print(f'===== Swapped =====')
pool_b.check()

Pool A status
DAI: 400
ETH: 1200
Price: 3.0
k: 480000

Pool B status
DAI: 2000
ETH: 6000
Price: 3.0
k: 12000000

Test Case 1
===== Swapped =====
Pool A status
DAI: 500.3
ETH: 960.0
Price: 1.918849
k: 480288.0

Test Case 2
===== Swapped =====
Pool A status
DAI: 414.041379
ETH: 1160.6
Price: 2.803101
k: 480536.424828

Test Case 3
===== Swapped =====
Pool B status
DAI: 1983.471074
ETH: 6050.15
Price: 3.050284
k: 12000297.520661



<b> 4. Write a test cases to benchmark non-blocking calculation.

In [10]:
import time

pool_a = PoolStatus('A', 400, 1200)
pool_b = PoolStatus('B', 2000, 5700)

pool_a.check()
pool_b.check()

s = time.perf_counter()

## Check if there is an arbitrage opportunity if I want to swap 10 DAI
opportunity = await calculate_arbitrage(pool_a, pool_b, output_dai=10)
opportunityStatus(opp=opportunity, output_dai=10)

## Since there is an opportunity, proceed with swap using the required amount of ETH required for 10 DAI
await pool_b.swap(input_eth=opportunity[0])
print(f'===== Swapped =====')
pool_b.check()

## Check again for arbitrage opportunity, this time for 20 DAI
opportunity = await calculate_arbitrage(pool_a, pool_b, output_dai=10)
opportunityStatus(opp=opportunity, output_dai=20)

## Again, proceed with swap by calculating amount of ETH required for 20 DAI
await pool_b.swap(input_eth=opportunity[0])
print(f'===== Swapped =====')
pool_b.check()

## Check again for arbitrage opportunity, back to 10 DAI
opportunity = await calculate_arbitrage(pool_a, pool_b, output_dai=10)
opportunityStatus(opp=opportunity, output_dai=10)

elapsed = time.perf_counter() - s
print(f"Executed in {elapsed:0.10f} seconds.")

Pool A status
DAI: 400
ETH: 1200
Price: 3.0
k: 480000

Pool B status
DAI: 2000
ETH: 5700
Price: 2.85
k: 11400000

--- Arbitrage opportunity exists ---
Using 10 DAI:
Buy 28.729146 ETH from Pool B
Price per DAI: 2.872915 ETH
Total price: 28.729146 ETH
Total profit: $0.539147

===== Swapped =====
Pool B status
DAI: 1989.97015
ETH: 5728.815333
Price: 2.878845
k: 11400171.510427

--- Arbitrage opportunity exists ---
Using 20 DAI:
Buy 29.020649 ETH from Pool B
Price per DAI: 1.451032 ETH
Total price: 29.020649 ETH
Total profit: $0.247644

===== Swapped =====
Pool B status
DAI: 1979.940302
ETH: 5757.923044
Price: 2.90813
k: 11400343.887882

--- No arbitrage opportunity ---

Executed in 0.0021730000 seconds.
